In [ ]:
#
%matplotlib inline
import matplotlib
import matplotlib_inline.backend_inline
import arviz as az
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
matplotlib_inline.backend_inline.set_matplotlib_formats("svg", "pdf", "retina")  # For export
import sys 
sys.path.append("../src/")
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
from cycler import cycler
az.style.use(["science"])
nice_fonts = {
        # Use LaTeX to write all text
        "font.family": "serif",
        # Always save as 'tight'
        "savefig.bbox" : "tight",
        "savefig.pad_inches" : 0.05,
        "ytick.right" : True,
        "font.serif" : "Times New Roman",
        "mathtext.fontset" : "dejavuserif",
        "axes.labelsize": 15,
        "font.size": 15,
        # Make the legend/label fonts a little smaller
        "legend.fontsize": 12,
        "xtick.labelsize": 12,
        "ytick.labelsize": 12,
        # Set line widths
        "axes.linewidth" : 0.5,
        "grid.linewidth" : 0.5,
        "lines.linewidth" : 1.,
        # Remove legend frame
        "legend.frameon" :False,
        # color-blind friendly cycle designed using https://colorcyclepicker.mpetroff.net/
        # see preview and check for colorblindness here https://coolors.co/107591-00c0bf-f69a48-fdcd49-8da798-a19368-525252-a6761d-7035b7-cf166e
        'axes.prop_cycle': cycler(color=['#107591','#00c0bf','#f69a48','#fdcd49','#8da798','#a19368','#525252','#a6761d','#7035b7','#cf166e'])
         
}
matplotlib.rcParams.update(nice_fonts)
az.style.use(["science",  "arviz-doc", 'tableau-colorblind10'])

In [ ]:
from utils.data import DatasetObjective
from darts import TimeSeries
from baselines.backtesting import BacktestingForecast

In [ ]:
from utils.hparams import  hparams
from utils.load_data import load_uk_data, load_substation_data
from utils.hparams import hparams
data_PT = load_substation_data(add_ghi=True).select_dtypes(exclude='object')
hparams.update({'window_size':96})
hparams.update({'horizon':48})
hparams.update({'encoder_type':'SeasonalNaive'})
hparams.update({'time_varying_known_categorical_feature': ['DAYOFWEEK', 'DAY', 'HOUR', 'Session']}),
hparams.update({'time_varying_known_feature': ['Ghi']})
hparams.update({'time_varying_unknown_feature': ['NetLoad-Ghi']})
hparams.update({'targets':['NetLoad']}) #'Load',
hparams.update({'latent_size': 256, 'depth': 2, 'batch_size': 64, 'num_head': 4, 'dropout': 0.1786342451901633,
                'alpha': 0.1786342451901633, 'emb_size': 32, 'embed_type': 'RotaryEmb', 'comb_type':'weighted-comb',
                })

In [ ]:
import scipy.signal as signal
import plotly.graph_objects as go
import plotly.express as px
data=data_PT['2018-8':'2022'].copy()
data=data.reset_index().drop_duplicates(subset='timestamp')
data=data.drop(columns=[ 'index'])

columns = list(data.columns)[1:]
data['timestamp'] = pd.to_datetime(data['timestamp'], utc=True).dt.tz_localize(None)
data=TimeSeries.from_dataframe(data, 'timestamp', columns, fill_missing_dates=True).pd_dataframe()

In [ ]:
n_rows=1
fig = px.scatter(data.reset_index(), x='timestamp', y='NetLoad')
fig.update_xaxes(matches=None, showticklabels=True, visible=True)
fig.update_layout(margin=dict(l=60, r=10, t=20, b=50))
fig.update_layout(template="plotly_white", font=dict(size=10))
fig.update_annotations(font_size=10)
fig.update_layout(autosize=True, height=150 * n_rows)
fig.show()                     

## Experiments one compare different metrics

In [ ]:
forecast_len=6 
incremental_len = 2
min_train_len=12
seed=777
max_epochs=50    
n_splits=10                 
window_type='expanding'        
dataset="PT",
exp_name="PT-Benchmark"
hparams.update({'max_epochs':max_epochs})
hparams.update({'max_epochs':max_epochs})
hparams.update({'num_trials':50}) 
file_name=f'{exp_name}_{window_type}' 
bactesting= BacktestingForecast(hparams,  data,  file_name,
                                    n_splits=n_splits, forecast_len = forecast_len, 
                                    incremental_len = incremental_len, min_train_len=min_train_len,
                                    window_type=window_type, 
                                    max_epochs=max_epochs)

In [ ]:
list_models=[ 'LSTM', 'MSTL' ,'CATBOOST', 'MSTL', 'SeasonalNaive', 'RF', 'LREGRESS', 'PatchTST','TimesNet', 'FEDformer']
for encoder_type in list_models: 
    hparams.update({'encoder_type':encoder_type})
    bactesting.fit(hparams, autotune=True)